In [13]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import progressbar

In [3]:
data = pd.read_csv('./erdata.csv', sep=';')
data.head(5)

,id,authors,venue,year,title
0,1,QD Inc,"San Diego,",NaN,11578 Sorrento Valley Road
1,2,"AS Argon, JG Hannoosh","Phil. Mag,",NaN,Initiation of crazes in polystyrene
2,3,"GH Hansen, LL Wetterberg, H SjÃ¶strÃ¶m, O NorÃ©n","The Histochemical Journal,",1992.0,Immunogold labelling is a quantitative method ...
3,4,"TM Hammett, P Harmon, W Rhodes",see,NaN,The Burden of Infectious Disease Among Inmates...
4,5,JR Cogdell,"NEW DIRECTIONS FOR TEACHING AND LEARNING,",1995.0,The Role of Faculty Advising in Science and En...


In [4]:
# Fill the NAs with empty strings to allow concatentation
data.authors = data.authors.fillna("")
data.venue = data.venue.fillna("")
data.year = data.year.astype(str)
data.year = data.year.fillna("")
data.title = data.title.fillna("")

In [5]:
data["fullText"] = data["authors"] + " " + data["venue"]+ " " + data["year"]+ " " + data["title"]


In [6]:
# Let's make sure we have no NAs
sum(data.fullText.isna())

0

### Create the blocks / Vectorizer
<br>
For this solution we are using SckitLearn's [Vectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html), which allows us to extract features from text like inputs. <br>
In order to do that we are transforming the data to strings, and concatenating it to a new column, later passed for extraction. <br>
The process generates a 2D array, with each row referencing an entity from the original Data passed, and each column having 1 if a respective feature belongs to this entity, 0 otherwise. 

In [7]:
# By default Vectorizer transforms strings to lowercase ==> https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data["fullText"])
feat = vectorizer.get_feature_names_out()
xArray = X.toarray()

In [9]:
print("Total Features found: " + str(len(feat)) + "\n\n")
print("Sample output:")
print(feat[2000:2100])

Total Features found: 92603


Sample output:
['abstinence' 'abstinent' 'abstr' 'abstrac' 'abstract' 'abstracted'
 'abstracting' 'abstraction' 'abstractions' 'abstractness' 'abstracts'
 'abstractâ' 'abstracuon' 'absurd' 'absurdities' 'abt' 'abts' 'abu'
 'abugov' 'abuknesha' 'abundance' 'abundances' 'abundant' 'abuse' 'abused'
 'abuser' 'abusers' 'abuses' 'abusive' 'abuzenadah' 'abwasser' 'aby'
 'abyssal' 'abyzov' 'abã' 'ac' 'ac0' 'ac0851' 'ac095' 'aca' 'acad'
 'academe' 'academia' 'academic' 'academically' 'academician'
 'academicpress' 'academics' 'academicâ' 'academies' 'academy' 'acadian'
 'acadimic' 'acanthopterygii' 'acar' 'acarine' 'acarol' 'acarregui'
 'acaseforredundantarraysofinexpensive' 'acc' 'accapp03' 'accc' 'acce'
 'accelaration' 'accelerate' 'accelerated' 'accelerating' 'acceleration'
 'accelerator' 'accelerators' 'accelerograms' 'accelerometer'
 'accelerometers' 'accelrys' 'accents' 'accentuated' 'accept'
 'acceptability' 'acceptable' 'acceptance' 'accepted' 'accepting'


In [12]:
def findElements(xArray,data, i):
    objList = []
    
    for j in range(len(xArray)):
        itemVector = xArray[j]
        
        # Check if the item contains the feature, if yes, append its ID
        if itemVector[i] == 1:
            objList.append(data.iloc[j].id)
    
    return objList

In [14]:
# Let's create a structure containing the blocks
# The format will be FeatureName : [Array Of Item IDs that contain the Feature]
bar = progressbar.ProgressBar(maxval=len(feat), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

dictMap = {}
for i in range(len(feat)):
    bar.update(i)
    dictMap[feat[i]] = findElements(xArray, data, i)

bar.finish()

[========================================================================] 100%


In [29]:
# Remove blocks with < 2 size
dictKeys = [x for x in dictMap.keys()]
for blk in dictKeys:
    if len(dictMap[blk]) < 2:
        del dictMap[blk]

In [38]:
# Let's print an example of the 'academics' feature
dictMap['academics']

[1527, 15546, 30821, 33340]

In [88]:
print(len(X.toarray()))
print(len(data))
print(len(X.toarray()[0]))
print(len(feat))

66879
66879
92603
92603


### Calculate comparisons

In [43]:
# In this exmaple we know the entities' structure. So each comparison would be comparing EntityA FieldA w/ EntityB FieldA, etc. 
# This let's us assume that a comparison between two entities would cost us 4 field comparisons. 
# If we did not know the schema, we would need to to compare all fields between each other. This brings up the number to #NumberOfEntityAKeys x #NumberOfEntityBKeys. E.g. Entity A has 4 fields and Entity B has 5 -> A comparison would be 4x5=20

compCost = 4
sum=0
for blk in dictMap:
    for i in range(len(blk)):
        sum = sum + compCost*i
print("Total Comparisons: " + str(sum))

Total Comparisons: 3779840


### Meta Building Graph
In order to build a graph, we are creating a series of edges in the form of 
[NodeFrom, NodeTo, Weight] <br>

A target Node is a node that exists in the same block as the original Node <br>
In order to build this graph we will:<br>
* Iterate over the dictionary we built earlier
* For each of its elements add edges and weights between the Entities (0 if it's the first occurence, +1 if it already exists)
<!-- end of the list -->
This allows us to create a graph faster than a double for loop<br>
There is one issue here: When in one block contains nodes 123 and 321 and another one contains 321 and 123 then with this implementation there will be an arc created for 123 -> 321 and when the second block is checked, the new arc 321->123 will be created, instead of updating the first one. <br>
How do we easily solve this? - By sorting the blocks' arrays / Thankfully, since the package worked iteratively and the initial Entities were sorted, the blocks had sorted values appended

In [34]:
dictMap['academics']

[1527, 15546, 30821, 33340]

In [41]:
class Graph:
# Constructor
    def __init__(self, num_of_nodes, directed=False):
        self.m_num_of_nodes = num_of_nodes
        self.m_directed = directed
        
        # Different representations of a graph
        self.m_list_of_edges = []
	
    # Add edge to a graph
    def add_edge(self, node1, node2, weight=1):        
        # Add the edge from node1 to node2 - If the edge exists, increase the weight by 1
        found = 0
        for i in range(len(self.m_list_of_edges)):
            if((self.m_list_of_edges[i][0]==node1) & (self.m_list_of_edges[i][0]==node2) ):
                found=1
                self.m_list_of_edges[i][2] = self.m_list_of_edges[i][2] + 1
        if(found == 0):
            self.m_list_of_edges.append([node1, node2, weight])
        
    def get_edges(self):
        return self.m_list_of_edges
    
    def set_edges(self, edges):
        self.m_list_of_edges = edges

	# Print a graph representation
    def print_edge_list(self, thres=0):
        num_of_edges = len(self.m_list_of_edges)
        if(thres!=0):
            num_of_edges = thres
        for i in range(thres):
            print("edge ", i+1, ": ", self.m_list_of_edges[i])

In [ ]:
graph = Graph(len(data))

# loop over all keys
for blk in dictMap:

    # for each key loop over the entity IDs
    for i in range(len(blk)):
        for j in range(i+1, len(blk)-1):
            graph.add_edge(blk[i], blk[j])
 

In [ ]:
# Remove edges w/ Weight < 2
edges = []
for edge in graph.get_edges():
    if edge[2] >= 2:
        edges.append(edge) 

graph.set_edges(edges)

In [ ]:
# The new number of comparisons is: NumberOfGraphEdges x ComparisonCost
compCost = 4

print("Total Comparisons: " + str(len(graph.get_edges() * 4)))

### Jaccard Similarity

In [ ]:
def JaccardSimilarity(title1, title2):
    print("Comparing: ",title1, title2 )
    title1Words = title1.split()
    title2Words = title1.split()
    union = list(set(title1Words) + set(title2Words))
    inter = set(title1Words).intersection(title2Words)
    return (union / inter)

In [ ]:
# An example

JaccardSimilarity(data[3, "title"], data[4, "title"])